In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from datetime import datetime
from sklearn.model_selection import train_test_split
import gc
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import pickle
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from xgboost import plot_importance
from sklearn.tree import DecisionTreeRegressor

In [ ]:
class ml_project:
    def __init__(self):
        pass
    def preprocess_data(self,tdata,store_data):
        data=pd.merge(tdata,store_data,on="Store")
        data.fillna(0,inplace=True)
        data['Promo2SinceWeek'] = data.Promo2SinceWeek.astype(int)
        data['Promo2SinceYear'] = data.Promo2SinceYear.astype(int)
        data['PromoInterval'] = data.PromoInterval.astype(str)
        data['CompetitionOpenSinceMonth'] = data.CompetitionOpenSinceMonth.astype(int)
        data['CompetitionOpenSinceYear'] = data.CompetitionOpenSinceYear.astype(int)
        assortment_mapping={'a':0,'b':1,'c':2}
        store_mapping={'a':0,'b':1,'c':2,'d':4}
        data.replace(assortment_mapping,inplace=True)
        data.replace(store_mapping,inplace=True)
    
        data['Date'] = pd.to_datetime(data['Date'])
    
        data['Year'] = data.Date.dt.year
        data['Month'] = data.Date.dt.month
        data['Day'] = data.Date.dt.day
        data['DayOfWeek'] = data.Date.dt.dayofweek
        data['WeekOfYear'] = data.Date.dt.weekofyear
        #calculation of time when the competition has opened in months
        data['CompetitionOpenMonths'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + (data.Month - data.CompetitionOpenSinceMonth)
        data['CompetitionOpenMonths'] = data.CompetitionOpenMonths.apply(lambda x: x if x > 0 else 0)
        data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
        data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    
        month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
        data['monthStr'] = data.Month.map(month2str)
        data['IsPromoMonth'] = 0
        count=0
        for interval in data.PromoInterval.unique():
            if interval!="":
                for month in interval.split(","):
                    data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1
        data['Date'] =  pd.to_datetime(train_data['Date'], format='%Y-%m-%d')
        data['DayOfYear']=data['Date'].dt.dayofyear
        data.loc[data["Promo2"]==0,"PromoOpen"]=0
        data=data.loc[data["Open"]==1]
        #data=data.drop(columns=['Unnamed: 0'])
        return data
    def set_train_data(self,train_data):
        self.train_data=train_data
    def set_test_data(self,test_data):
        self.test_data=test_data
    def set_features(self,features):
        self.features=features
    def train_random_forest_regressor(self):
        gc.collect()
        model=RandomForestRegressor(max_depth=50,max_features='auto',min_samples_leaf=4,min_samples_split=10,n_estimators=400)
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        model.fit(x_train,y_train)
        self.random_forest_regressor=model
    def train_random_forest_regressor_default(self):
        model=RandomForestRegressor()
        gc.collect()
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        model.fit(x_train,y_train)
        self.random_forest_regressor_default=model
    def train_linear_regressor(self):
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        model=LinearRegression()
        gc.collect()
        model.fit(x_train,y_train)
        self.linear_regressor=model
    def train_xgboost_regressor(self):
        x_train,x_val,y_train,y_val=train_test_split(train_data[features],train_data[['Sales']],random_state=200,test_size=0.2)
        model=xgb.XGBRegressor(objective="reg:linear",booster="gbtree", eta=0.03,max_depth=10, subsample=0.9, 
                  colsample_bytree=0.7,silent=1,seed=10)
        gc.collect()
        model.fit(x_train,y_train)
        self.xgb_regressor=model
    def train_decision_tree_regressor(self):
        x_train,x_val,y_train,y_val=train_test_split(train_data[features],train_data[['Sales']],random_state=200,test_size=0.2)
        model = DecisionTreeRegressor(max_depth=20)
        gc.collect()
        model.fit(x_train,y_train)
        self.decision_tree_regressor=model
    def set_random_forest_regressor(self,model):
        self.random_forest_regressor=model
    def set_random_forest_regressor_default(self,model):
        self.random_forest_regressor_default=model
    def set_linear_regressor(self,model):
        self.linear_regressor=model
    def set_xgb_regressor(self,model):
        self.xgb_regressor=model
    def set_decision_tree_regressor(self,model):
        self.decision_tree_regressor=model
    def get_random_forest_validation_score(self):
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        predictions = self.random_forest_regressor.predict(x_val)
        rmse = sqrt(mean_squared_error(y_val,predictions))
        return rmse
    def get_random_forest_validation_score_default(self):
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        predictions = self.random_forest_regressor_default.predict(x_val)
        rmse = sqrt(mean_squared_error(y_val,predictions))
        return rmse
    def get_linear_regressor_validation_score(self):
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        LinearModelPredictions = self.linear_regressor.predict(x_val)
        rmse = sqrt(mean_squared_error(y_val, LinearModelPredictions))
        return rmse
    def get_xgb_regressor_validation_score(self):
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        predictions = self.xgb_regressor.predict(x_val)
        rmse = sqrt(mean_squared_error(y_val, predictions))
        return rmse
    def get_decision_tree_regressor_validation_score(self):
        x_train,x_val,y_train,y_val=train_test_split(self.train_data[features],self.train_data[['Sales']],random_state=200,test_size=0.2)
        predictions = self.decision_tree_regressor.predict(x_val)
        rmse = sqrt(mean_squared_error(y_val, predictions))
        
        return rmse
    def pickle_decision_tree_regressor(self):
        filename="decision_tree_regressor.sav"
        pickle.dump(self.linear_regressor,open(filename,"wb"))
    def pickle_linear_regressor(self):
        filename="linear_regressor.sav"
        pickle.dump(self.linear_regressor,open(filename,"wb"))
    def pickle_xgb_regressor(self):
        filename="xgb_regressor.sav"
        pickle.dump(self.xgb_regressor,open(filename,"wb"))
    def pickle_random_forest_regressor(self):
        filename="random_forest_regressor.sav"
        pickle.dump(self.random_forest_regressor,open(filename,"wb"))
    def pickle_random_forest_regressor_default(self):
        filename="random_forest_regressor_default.sav"
        pickle.dump(self.random_forest_regressor_default,open(filename,"wb"))
    def plot_weights_random_forest_regressor(self):
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0] = 20
        fig_size[1] = 10
        plt.rcParams["figure.figsize"] = fig_size
        height=self.random_forest_regressor.feature_importances_
        bars=self.features
        y_pos = np.arange(len(bars))
        l=[1 for i in range(11)]
        li=np.array(l)
        plt.bar(y_pos,height)
        plt.xticks(y_pos, bars,rotation=90)
        plt.show()
    def plot_weights_random_forest_regressor_default(self):
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0] = 20
        fig_size[1] = 10
        plt.rcParams["figure.figsize"] = fig_size
        height=self.random_forest_regressor_default.feature_importances_
        bars=self.features
        y_pos = np.arange(len(bars))
        l=[1 for i in range(11)]
        li=np.array(l)
        plt.bar(y_pos,height)
        plt.xticks(y_pos, bars,rotation=90)
        plt.show()
    def plot_weights_linear_regressor(self):
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0] = 20
        fig_size[1] = 10
        plt.rcParams["figure.figsize"] = fig_size
        height=self.linear_regressor.coef_[0]
        bars=self.features
        y_pos = np.arange(len(bars))
        l=[1 for i in range(11)]
        li=np.array(l)
        plt.bar(y_pos,height)
        plt.ylim(top=500)  # adjust the top leaving bottom unchanged
        plt.ylim(bottom=0)
        plt.xticks(y_pos, bars,rotation=90)
        plt.show()
    def plot_weights_xgb_regressor(self):
        plot_importance(self.xgb_regressor)
    def plot_weights_decision_tree_regressor(self):
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0] = 20
        fig_size[1] = 10
        plt.rcParams["figure.figsize"] = fig_size
        height=self.decision_tree_regressor.feature_importances_
        bars=self.features
        y_pos = np.arange(len(bars))
        l=[1 for i in range(11)]
        li=np.array(l)
        plt.bar(y_pos,height)
        plt.xticks(y_pos, bars,rotation=90)
        plt.show()
    def generate_csv(self,model,filename):
        predictions = model.predict(self.test_data[self.features])
        predictions=predictions.reshape(len(predictions))
        result = pd.DataFrame({"Id": self.test_data["Id"], 'Sales':predictions})
        result.to_csv(filename)

In [ ]:
train_data=pd.read_csv('train.csv',low_memory=False)
test_data=pd.read_csv('test.csv',low_memory=False)
store_data=pd.read_csv('store.csv',low_memory=False)
obj=ml_project()
train_data=obj.preprocess_data(train_data,store_data)
test_data=obj.preprocess_data(test_data,store_data)
#train_data.to_csv("train_data_processed.csv")
#test_data.to_csv("test_data_processed.csv")

In [ ]:
#train_data=pd.read_csv("train_data_processed.csv")
#test_data=pd.read_csv("test_data_processed.csv")
features=['Store','Customers','Promo','StoreType','Assortment','CompetitionOpenMonths',
            'CompetitionDistance','DayOfWeek','Promo2','PromoOpen']

In [ ]:
obj.set_test_data(test_data)
obj.set_train_data(train_data)
obj.set_features(features)

In [ ]:
filename="linear_regressor.sav"
#pickle.dump(model,open(filename,"wb"))
model=pickle.load(open(filename,'rb'))
obj.set_linear_regressor(model)
print(obj.get_linear_regressor_validation_score())
obj.plot_weights_linear_regressor()
gc.collect()
obj.generate_csv(model,"LinearModel.csv")


In [ ]:
filename="decision_tree_regressor.sav"
#pickle.dump(model,open(filename,"wb"))
model=pickle.load(open(filename,'rb'))
obj.set_linear_regressor(model)
print(obj.get_linear_regressor_validation_score())
obj.plot_weights_linear_regressor()
gc.collect()
obj.generate_csv(model,"DecisionTree.csv")

In [ ]:
obj.train_xgboost_regressor()


#filename="xgb_regressor.sav"
#pickle.dump(model,open(filename,"wb"))
#model=pickle.load(open(filename,'rb'))
#obj.set_xgb_regressor(model)
print(obj.get_xgb_regressor_validation_score())
obj.plot_weights_xgb_regressor()
gc.collect()
obj.generate_csv(model,"XGBOOST.csv")

In [ ]:
filename="random_forest_regressor_default.sav"
#pickle.dump(model,open(filename,"wb"))
model=pickle.load(open(filename,'rb'))
obj.set_random_forest_regressor_default(model)
print(obj.get_random_forest_validation_score_default())
obj.plot_weights_random_forest_regressor_default()
gc.collect()
obj.generate_csv(model,"RandomForestDefault.csv")

In [ ]:
filename="random_forest_regressor.sav"
#pickle.dump(model,open(filename,"wb"))
model=pickle.load(open(filename,'rb'))
obj.set_random_forest_regressor(model)
print(obj.get_random_forest_validation_score())
obj.plot_weights_random_forest_regressor()
gc.collect()
obj.generate_csv(model,"RandomForest.csv")